
## **BERTopic**
BERTopic is a topic modeling technique that leverages 🤗 transformers and a custom class-based TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions. 

<br>


In [1]:
## install bertopic
!pip install bertopic

     ---------------------------------------- 14.7/14.7 MB 4.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.3
    Uninstalling numpy-1.19.3:
      Successfully uninstalled numpy-1.19.3


ERROR: Could not install packages due to an OSError: [WinError 5] 存取被拒。: 'C:\\Users\\pc127\\anaconda3\\Lib\\site-packages\\~-mpy\\.libs\\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



# Data

In [4]:
import pandas as pd
df =pd.read_csv("abstarct_data.csv")
df['abstract'] = df['abstract'].apply(eval)

In [9]:
data = df['abstract']
print(data[0])

['近年來', '電子商店', '已成', '一種', '新興', '零售', '通路', '這種', '伴隨', 'internet', '發展', '快速', '成長', '電子商務', '逐漸', '改變', '人類', '生活習慣', '創造', '商業', '行為', '並成', '近年來', '企業', '經營', '熱門話題', '國內外', '過去', '對於', '電子商店', '研究', '大部分', '集中', '涉入', '理論', '科技', '接受', '模式', '服務', '品質', '觀點', '探討', '電子商店', '消費者', '購買', '意願', '或是', '探討', '知覺', '風險', '信任', '消費者', '購買', '意願', '影響', '關係', '有關', '交易成本', '觀點', '探討', '電子商店', '線上', '購買', '意願', '議題', '絕大部分', '探討', '交易成本', '知覺', '風險', '消費者', '線上', '購買', '意願', '關係', '或是', '探討', '交易成本', '信任', '消費者', '線上', '購買', '意願', '關係', '鮮少', '研究', '交易成本', '知覺', '風險', '信任', '消費者', '網路', '購買', '意願', '四者', '相互影響', '關係', '深入探討', '因此', '研究', '目的', '探討', '交易成本', '消費者', '網路', '購買', '意願', '影響', '探討', '信任', '知覺', '風險', '消費者', '網路', '購買', '意願', '相互影響', '關係', '研究', '研究', '範圍', '針對', '上具', '規模', '電子商店', '為主', '針對', '有網', '使用', '經驗', '曾經', '瀏覽', '電子商店', '並且', '電子商店', '完成', '過的', '上班族', '學生', '主要', '研究', '對象', '並採', '便利', '抽樣', '運用', '一般', '問卷', '網路', '問卷', '進行', '調查', '回收', '問卷', '剔除', '無效', '問卷', '

In [10]:
docs = []
for d in data:
    docs.append(" ".join(d))

In [11]:
len(docs)

39009

# **Topic Modeling**

In this example, we will go through the main components of BERTopic and the steps necessary to create a strong topic model. 




## Training

We start by instantiating BERTopic. We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead. 

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model. 


In [12]:
from bertopic import BERTopic

topic_model = BERTopic(language="chinese (traditional)", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(docs)

Batches:   1%|          | 13/1220 [00:17<27:24,  1.36s/it]


KeyboardInterrupt: 

In [ ]:
import numpy as np
np.argmax(probs[0])
# topics[0]

: 

In [ ]:
import pickle 
pickle.dump((topics,probs),open("bert_topic1.pk","wb"))

: 

**NOTE**: Use `language="multilingual"` to select a model that support 50+ languages.

## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents. 

In [ ]:
import pickle 
topics,probs = pickle.load(open("bert_topic1.pk","rb"))

In [ ]:
freq = topic_model.get_topic_info(); freq.head(5)

: 

In [ ]:
topic_model

: 

-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [ ]:
topic_model.get_topic(5)  # Select the most frequent topic

: 

**NOTE**: BERTopic is stocastich which mmeans that the topics might differ across runs. This is mostly due to the stocastisch nature of UMAP.

# **Visualization**
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created. 

## Visualize Topics
After having trained our `BERTopic` model, we can iteratively go through perhaps a hundred topic to get a good 
understanding of the topics that were extract. However, that takes quite some time and lacks a global representation. 
Instead, we can visualize the topics that were generated in a way very similar to 
[LDAvis](https://github.com/cpsievert/LDAvis):

In [ ]:
topic_model.visualize_topics()

: 

In [ ]:
len(topic_model.topics)

: 

## Visualize Topic Probabilities

The variable `probabilities` that is returned from `transform()` or `fit_transform()` can 
be used to understand how confident BERTopic is that certain topics can be found in a document. 

To visualize the distributions, we simply call:

In [ ]:
topic_model.visualize_distribution(probs[200], min_probability=0.015)

: 

## Visualize Topic Hierarchy

The topics that were created can be hierarchically reduced. In order to understand the potential hierarchical structure of the topics, we can use scipy.cluster.hierarchy to create clusters and visualize how they relate to one another. This might help selecting an appropriate nr_topics when reducing the number of topics that you have created.

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=50)

: 

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [ ]:
topic_model.visualize_barchart(top_n_topics=5)

: 

## Visualize Topic Similarity
Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.

In [ ]:
topic_model.visualize_heatmap(n_clusters=20, width=1000, height=1000)

: 

## Visualize Term Score Decline
Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added. At some point adding words to the topic representation only marginally increases the total c-TF-IDF score and would not be beneficial for its representation.

To visualize this effect, we can plot the c-TF-IDF scores for each topic by the term rank of each word. In other words, the position of the words (term rank), where the words with the highest c-TF-IDF score will have a rank of 1, will be put on the x-axis. Whereas the y-axis will be populated by the c-TF-IDF scores. The result is a visualization that shows you the decline of c-TF-IDF score when adding words to the topic representation. It allows you, using the elbow method, the select the best number of words in a topic.


In [ ]:
topic_model.visualize_term_rank()

: 

# **Topic Representation**
After having created the topic model, you might not be satisfied with some of the parameters you have chosen. Fortunately, BERTopic allows you to update the topics after they have been created. 

This allows for fine-tuning the model to your specifications and wishes. 

## Update Topics
When you have trained a model and viewed the topics and the words that represent them,
you might not be satisfied with the representation. Perhaps you forgot to remove
stopwords or you want to try out a different `n_gram_range`. We can use the function `update_topics` to update 
the topic representation with new parameters for `c-TF-IDF`: 


In [ ]:
topic_model.update_topics(data, topics, n_gram_range=(1, 2))

: 

In [ ]:
topic_model.get_topic(0)   # We select topic that we viewed before

: 

## Topic Reduction
We can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so, 
is that you can decide the number of topics after knowing how many are actually created. It is difficult to 
predict before training your model how many topics that are in your documents and how many will be extracted. 
Instead, we can decide afterwards how many topics seems realistic:





In [ ]:
new_topics, new_probs = topic_model.reduce_topics(data, topics, probs, nr_topics=60)

: 

# **Search Topics**
After having trained our model, we can use `find_topics` to search for topics that are similar 
to an input search_term. Here, we are going to be searching for topics that closely relate the 
search term "vehicle". Then, we extract the most similar topic and check the results: 

In [ ]:
similar_topics, similarity = topic_model.find_topics("人工智慧", top_n=10); similar_topics

: 

In [ ]:
for topic in similar_topics:
    print(topic_model.get_topic(topic))

: 

In [ ]:
new_doc = "By semantically optimizing your content, you add more meaning to the words you use. You optimize for the true intent of your users, not just answering a simple query. This means answering the first question, then answering the second, third, fourth, and fifth questions right after that."

topic_model.transform([new_doc])

: 

# **Model serialization**
The model and its internal settings can easily be saved. Note that the documents and embeddings will not be saved. However, UMAP and HDBSCAN will be saved. 

In [ ]:
# Save model
topic_model.save("my_model")	

: 

In [ ]:
# Load model
my_model = BERTopic.load("my_model")	

: 

# **Embedding Models**
The parameter `embedding_model` takes in a string pointing to a sentence-transformers model, a SentenceTransformer, or a Flair DocumentEmbedding model.

## Sentence-Transformers
You can select any model from sentence-transformers here and pass it through BERTopic with embedding_model:



In [ ]:
topic_model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens")

: 

Or select a SentenceTransformer model with your own parameters:


In [ ]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens", device="cpu")
topic_model = BERTopic(embedding_model=sentence_model, verbose=True)

: 

Click [here](https://www.sbert.net/docs/pretrained_models.html) for a list of supported sentence transformers models.  
